# Continuous Control

In this notebook, I'll train an agent to solve the Reacher Environment using Unity ML-Agents.

---

# Index

- []()
- []()
- []()
- []()
- []()

## 1. Setup the Environment

<img align="left" width="150" src="https://www.nclouds.com/img/services/toolkit/sagemaker.png"/>

This notebook was developed on AWS SageMaker.

The kernel used is **conda_python3**

To setup this environment on SageMaker you need to run the next 3 cells.

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install -r requirements.txt

In [ ]:
#!pip install torch===1.5.1 torchvision===0.6.1 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip -q install ./python

## 2. Start the Environment

The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from unityagents import UnityEnvironment
from collections import deque
from agent import Agent

# One agent
#env_path = "one_agent/Reacher_Windows_x86_64/Reacher.exe"
env_path = "one_agent/Reacher_Linux/Reacher.x86_64"

# Multi agent
#env_path = "multi_agent/Reacher_Windows_x86_64/Reacher.exe"
#env_path = "multi_agent/Reacher_Linux/Reacher.x86_64"

env = UnityEnvironment(file_name=env_path, no_graphics=True)

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

## 3. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

## 4. Define helper functions to training session

Here we'll create a function that can be very helpful to teach the agent.

In [ ]:
def rl_trainer(agent, n_episodes=100, max_time_step=None, scores_deque=100, print_range=10, early_stop=13, add_noise=False, verbose=False):
    """Deep Q-Learning trainer.
    
    Params
    ======
        agent (object): The agent object
        n_episodes (int): maximum number of training episodes.
        max_time_step (int or None): The max time step per episode, if None, the agent will training until environment is done.
        scores_deque (int): The len of score deque.
        print_range (int): range to print partials results.
        early_stop (int): Stop training when achieve a defined score respecting 10 min n_episodes.
        add_noise (bool): If True, we'll reset at each step and add_noise at each action call.
        verbose (bool): If verbose true, we'll print some infos on console.
    """
    scores = [] # list containing scores from each episode
    scores_window = deque(maxlen=scores_deque) # last print_range scores
    
    env_info = env.reset(train_mode=True)[brain_name]
            
    for ep in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        score = 0
        time_step = 0
        training = True # control max_time_step
        
        if add_noise:
            agent.reset()
            
        while training:
            action = agent.act(state, add_noise=add_noise)
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0]
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            time_step += 1
            
            if isinstance(max_time_step, (int, float)):
                if time_step >= max_time_step:
                    training = False
            if done:
                break
                
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        if verbose:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(ep, np.mean(scores_window)), end="")
            if ep % print_range == 0:
                print('\rEpisode {}\tAverage Score: {:.2f}'.format(ep, np.mean(scores_window)))
            
        if np.mean(scores_window) >= early_stop:
            if verbose:
                print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(ep, np.mean(scores_window)))
            break
            
    return scores, ep, np.mean(scores_window)

## 5. Training the Agent

In the next code cells, we will train the Agent to work on environment.

In [ ]:
%%time

env_info = env.reset(train_mode=True)[brain_name] # reset the environment

agent = Agent(state_size=state_size, action_size=action_size, seed=399, lr_actor=1e-3, lr_critic=1e-3,
              memory_size=int(1e6), batch_size=128, gamma=0.98, tau=5e-3, weight_decay=0.0,
              actor_units=(256, 128), critic_units=(256, 128), action_min=-1, action_max=1)

scores, episodes, last_avg_score = rl_trainer(agent, n_episodes=2000, max_time_step=None,
                                              print_range=10, early_stop=30, add_noise=True, verbose=True)

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Average Score')
plt.xlabel('Episode #')
plt.show()

### 5.1 Agent losses visualization

In [ ]:
plt.plot(agent.losses)
plt.title("Losses")
plt.show()

### Save weights

In [ ]:
agent.save_model()

### Load weights

In [ ]:
agent.load_model()

## 6. Test Agent on Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance.

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]
state = env_info.vector_observations[0]
score = 0
eps = 0.05
while True:
    action = agent.act(state, eps)
    env_info = env.step(action)[brain_name]
    next_state = env_info.vector_observations[0]
    reward = env_info.rewards[0]
    done = env_info.local_done[0]
    score += reward
    state = next_state
    if done:
        break
    
print("Score: {}".format(score))

When finished, you can close the environment.

In [ ]:
env.close()